### <center> SEED Labs – Return-to-libc Attack
    
The content of this lab notebook is based on the SEED Labs by Dr. Wenliang Du, Syracuse University. The original copyright passage and license are retained.

Copyright © 2006 - 2016 Wenliang Du, Syracuse University.

The development of this document was partially funded by the National Science Foundation under Award
No. 1303306 and 1318814. This work is licensed under a Creative Commons Attribution - Non Commercial Share Alike
4.0 International License. A human-readable summary of (and not a substitute for) the license is
the following: You are free to copy and redistribute the material in any medium or format. You must give
appropriate credit. If you remix, transform, or build upon the material, you must distribute your contributions
under the same license as the original. You may not use the material for commercial purposes.

### 1. Lab Overview
The learning objective of this lab is for students to gain the first-hand experience on an interesting variant of
buffer-overflow attack; this attack can bypass an existing protection scheme currently implemented in major
Linux operating systems. 

A common way to exploit a buffer-overflow vulnerability is to overflow the buffer
with a malicious shellcode, and then cause the vulnerable program to jump to the shellcode that is stored in
the stack. To prevent these types of attacks, some operating systems allow system administrators to make
stacks non-executable; therefore, jumping to the shellcode will cause the program to fail.

Unfortunately, the above protection scheme is not fool-proof; there exists a variant of buffer-overflow
attack called the return-to-libc attack, which does not need an executable stack; it does not even use
shell code. Instead, it causes the vulnerable program to jump to some existing code, such as the system()
function in the libc library, which is already loaded into the memory.

In this lab, students are given a program with a buffer-overflow vulnerability; their task is to develop a
return-to-libc attack to exploit the vulnerability and finally to gain the root privilege. In addition to
the attacks, students will be guided to walk through several protection schemes that have been implemented
in Ubuntu to counter against the buffer-overflow attacks. Students need to evaluate whether the schemes
work or not and explain why.

In [ ]:
# setup temporary directory
!mkdir tmp

### 2. Lab Tasks

### 2.1 Initial Setup
You can execute the lab tasks using our pre-built Ubuntu virtual machines. Ubuntu and other Linux
distributions have implemented several security mechanisms to make the buffer-overflow attack difficult. To
simply our attacks, we need to disable them first.

#### Address Space Randomization: 

Ubuntu and several other Linux-based systems uses address space randomization
to randomize the starting address of heap and stack. This makes guessing the exact addresses
difficult; guessing addresses is one of the critical steps of buffer-overflow attacks. In this lab, we disable
these features using the following commands:

```
$ sudo sysctl -w kernel.randomize_va_space=0
```

#### The StackGuard Protection Scheme:

The GCC compiler implements a security mechanism called **Stack Guard** to prevent buffer overflows. In the presence of this protection, buffer overflow will not work. You
can disable this protection if you compile the program using the -fno-stack-protector switch. For example,
to compile a program example.c with Stack Guard disabled, you may use the flag `-fno-stack-protector` with your `gcc` statement. 

Because the objective of this lab is to show that the non-executable stack protection does not work, you
should always compile your program using the `-z noexecstack` option in this lab.

In [ ]:
%%writefile tmp/lab3_retlib.c
/* This program has a buffer overflow vulnerability. */
/* Our task is to exploit this vulnerability */
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
int bof(FILE *badfile){
  char buffer[12];
  /* The following statement has a buffer overflow problem */
  fread(buffer, sizeof(char), 40, badfile);
  return 1;
}

int main(int argc, char **argv){
  FILE *badfile;
  badfile = fopen("badfile", "r");
  bof(badfile);
  printf("Returned Properly\n");
  fclose(badfile);
  return 1;
}

In a terminal, compile the above vulnerable program and make it set-uid. Copy and paste the commands that you use in the cell below

In [ ]:
# copy commands here

The above program has a buffer overflow vulnerability. It first reads an input of size 40 bytes from
a file called **badfile** into a buffer of size 12, causing the overflow. The function `fread()` does not check
boundaries, so buffer overflow will occur. Since this program is a set-root-uid program, if a normal user
can exploit this buffer overflow vulnerability, the normal user might be able to get a root shell. 

It should be noted that the program gets its input from a file called **badfile**. This file is under users’ control. Now, our
objective is to create the contents for **badfile**, such that when the vulnerable program copies the contents
into its buffer, a root shell can be spawned.

#### 2.3 Task 1: Exploiting the Vulnerability

Create the badfile. You may use the following framework to create one.

In [ ]:
%%writefile tmp/lab3_exploit.c
/* exploit.c */
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
int main(int argc, char **argv){
  char buf[40];
  FILE *badfile;
  badfile = fopen("./badfile", "w");
  /* You need to decide the addresses and the values for X, Y, Z. The order of the following
     three statements does not imply the order of X, Y, Z.
     Actually, we intentionally scrambled the order. */
  *(long *) &buf[X] = some address ; // "/bin/sh"
  *(long *) &buf[Y] = some address ; // system()
  *(long *) &buf[Z] = some address ; // exit()

  fwrite(buf, sizeof(buf), 1, badfile);
  fclose(badfile);
}

You need to figure out the values for those addresses, as well as to find out where to store those addresses.
If you incorrectly calculate the locations, your attack might not work.
After you finish the above program, compile and run it; this will generate the contents for **badfile**. Run
the vulnerable program retlib. If your exploit is implemented correctly, when the function bof returns, it
will return to the `system()` libc function, and execute `system("/bin/sh")`. If the vulnerable program
is running with the root privilege, you can get the root shell at this point. It should be noted that the `exit()` function is not very necessary for this attack; however, without this function, when `system()` returns, the program might crash, causing suspicions. 

In a terminal, compile the above exploit and demonstrate that your attack works. Copy your commands in the cell below

In [ ]:
# copy commands here

####  Questions. In the cell belowt, please answer the following questions:

- Please describe how you decide the values for X, Y and Z. Either show us your reasoning, or if you use trial-and-error approach, show your trials.

- After your attack is successful, change the file name of retlib to a different name, making sure that the length of the file names are different. For example, you can change it to newretlib. Repeat the attack (without changing the content of badfile). Is your attack successful or not? If it does not succeed, explain why.

#### Answers:

